In [0]:
%%sh
aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/audience_demographics/version=1.0.0/range_type=MONTH/date=2018-01-31/




In [0]:

seg_by_demographics_audience="s3://aardvark-prod-pdx-mdm-to-int/audience_demographics/version=1.0.0/range_type=MONTH/date=2018-01-31/"
df=spark.read.option("basePath", "s3://aardvark-prod-pdx-mdm-to-int/audience_demographics/version=1.0.0").parquet(seg_by_demographics_audience)
# df.groupBy('device_id').count().show()
# df.groupBy('kpi').count().show()
# df.show(10,False)
df.printSchema()

In [0]:

import datetime
from aadatapipelinecore.core.fs.driver import unified
from aadatapipelinecore.core.fs.driver.unified import UnifiedDTO
from aadatapipelinecore.core.log import logger
from aadatapipelinecore.core.pipeline import schema, type_
from aadatapipelinecore.core.urn import Urn
from pyspark.sql.functions import lit, coalesce,upper
from aadatapipelinecore.core.utils.identifier import atomic_id
def main(spark, params):
    start = datetime.datetime.strptime(params.get("start"), "%Y-%m-%d")
    end = datetime.datetime.strptime(params.get("end"), "%Y-%m-%d")
    granularities = params.get("granularity", "daily").split(",")
    original_namespace = params.get("original_namespace")
    target_namespace = params.get("target_namespace")
    urn = Urn(
        namespace=target_namespace,
        deduplicate=True,
        identifier=atomic_id()
    )
    urn.save_mode = "overwrite"
    urn.bypass_data_monitor = True
    urn.coalesce = 1
    urn.manipulation = "insert"
    urn.event = "transform"
    d = end
    logger.info(granularities)
    while d > start:
        d_str = d.strftime("%Y-%m")
        for granularity in granularities:
            logger.info("for granularity: {}".format(granularity))
            logger.info("reading granularity = {} and d_str = {} from {} ".format(
                granularity, d_str, original_namespace))
            try:
                usage_df = (
                    spark
                        .read
                        .option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified"
                                            "/{}/fact/".format(original_namespace))
                        .parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/{}/fact/"
                                 "granularity={}/date={}*".format(original_namespace, granularity, d_str))
                )
            except Exception as e:
                logger.error(e.message)
                logger.error("date may not exist: {}-{}".format(granularity, d_str))
                continue

            usage_df=(
                usage_df
                    .withColumn('country_code', upper(usage_df['country_code']))
                    .withColumnRenamed('app_id', 'product_key')
                    .withColumnRenamed('city_id', 'city_key')
                    .withColumn('est_average_session_duration_milliseconds', usage_df['est_average_time_per_user'] / usage_df['est_average_session_per_user'])
                    .withColumnRenamed('est_average_time_per_user_milliseconds', 'est_average_time_per_user')
                    .withColumnRenamed('est_active_users', 'est_average_active_users')
                    .withColumn('est_average_bytes_per_session', usage_df['est_average_bytes_per_user'] / usage_df['est_average_session_per_user'])
                    .withColumn('est_total_time', usage_df['est_average_active_users'] * usage_df['est_average_time_per_user'])
            )

            usage_df.show()
            d -= datetime.timedelta(days=31)
            
            usage_df.createOrReplaceTempView("df")
            all_df = usage_df.drop("_identifier")
            all_df.createOrReplaceTempView("df")
            all_df=spark.sql("""
                SELECT product_key,city_key,est_affinity,est_average_active_users,est_average_bytes_per_session,est_average_bytes_per_user,est_average_session_duration,est_average_session_per_user,est_average_time_per_user,est_total_time,est_usage_penetration,est_average_session_duration_milliseconds,
                CASE WHEN granularity='m' THEN 'monthly' 
                    WHEN granularity='w' THEN 'weekly' END AS granularity,
                    date,
                    device_code,
                    country_code
                FROM df 
                WHERE est_average_active_users is not NULL
            """)
            all_df.createOrReplaceTempView("df")
            all_df.show()

params_list={
    'start':'2020-8-30',
    'end':'2020-8-31',
    'granularity':'m',
    'original_namespace':'usage.city-level.v3',
    'target_namespace':'aa.usage_city.v6'
    
}

main(spark,params=params_list)

In [0]:

import datetime
from aadatapipelinecore.core.fs.driver import unified
from aadatapipelinecore.core.fs.driver.unified import UnifiedDTO
from aadatapipelinecore.core.log import logger
from aadatapipelinecore.core.pipeline import schema, type_
from aadatapipelinecore.core.urn import Urn
from pyspark.sql.functions import lit, coalesce,upper
from aadatapipelinecore.core.utils.identifier import atomic_id
def main(spark, params):
    start = datetime.datetime.strptime(params.get("start"), "%Y-%m-%d")
    end = datetime.datetime.strptime(params.get("end"), "%Y-%m-%d")
    granularities = params.get("granularity", "daily").split(",")
    original_namespace_age_gender = params.get("original_namespace_age_gender")
    original_namespace_audience = params.get("original_namespace_audience")
    target_namespace = params.get("target_namespace")
    urn = Urn(
        namespace=target_namespace,
        deduplicate=True,
        identifier=atomic_id()
    )
    urn.save_mode = "overwrite"
    urn.bypass_data_monitor = True
    urn.coalesce = 1
    urn.manipulation = "insert"
    urn.event = "transform"
    d = end
    logger.info(granularities)
    while d > start:
        d_str = d.strftime("%Y-%m")
        for granularity in granularities:
            logger.info("for granularity: {}".format(granularity))
            logger.info("reading granularity = {} and d_str = {} from {} ".format(
                granularity, d_str, original_namespace_age_gender))
            logger.info("reading granularity = {} and d_str = {} from {} ".format(
                granularity, d_str, original_namespace_audience))
            try:
                usage_df_age_gender = (
                    spark
                        .read
                        .option("basePath","s3://b2c-prod-data-pipeline-unified-usage/unified/{}/fact/".format(original_namespace_age_gender))
                        .parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/{}/fact/granularity={}/month={}".format(original_namespace_age_gender, granularity, d_str))
                )
                usage_df_audience = (
                    spark
                        .read
                        .option("basePath", "s3a://b2c-prod-data-pipeline-unified-usage"
                                            "/unified/{}/fact/".format(original_namespace_audience))
                        .parquet("s3a://b2c-prod-data-pipeline-unified-usage/unified/{}/fact/"
                                 "granularity={}/month={}".format(original_namespace_audience, granularity, d_str))
                )
            except Exception as e:
                logger.error(e.message)
                logger.error("date may not exist: {}-{}".format(granularity, d_str))
                continue

            usage_df_age_gender.show()
            
            usage_df_age_gender = usage_df_age_gender.groupby(usage_df_age_gender.device_id, usage_df_age_gender.store_id, usage_df_age_gender.date, usage_df_age_gender.app_id,usage_df_age_gender.gender,usage_df_age_gender.age).pivot("kpi").max("estimate")

            usage_df_age_gender = (
                usage_df_age_gender
                .withColumnRenamed('gender', 'gender_key')
                .withColumnRenamed('age', 'age_key')
                .withColumnRenamed('app_id', 'product_key')
                .withColumnRenamed('5', 'est_average_active_days')
                .withColumnRenamed('3', 'est_average_session_duration_milliseconds')
                .withColumnRenamed('2', 'est_average_session_per_user')
                .withColumnRenamed('8', 'est_average_time_per_user_milliseconds')
                .withColumnRenamed('1', 'est_active_users')
                .withColumnRenamed('4', 'est_install_penetration')
                .withColumnRenamed('17', 'est_average_bytes_per_session')
                .withColumnRenamed('7', 'est_average_bytes_per_user')
                .withColumnRenamed('20', 'est_percentage_of_bytes_wifi_in_total')
                .withColumnRenamed('10', 'est_open_rate')
                .withColumnRenamed('6', 'est_percentage_of_active_days_in_granularity')
                .withColumnRenamed('9', 'est_usage_penetration')
                .withColumnRenamed('23', 'est_install_base')
            )
            usage_df_age_gender = usage_df_age_gender.drop("21") 
            usage_df_age_gender.createOrReplaceTempView("df_ag")
            usage_df_age_gender = spark.sql("""
                SELECT 
                df_ag.*,
                CASE WHEN device_id='1001' THEN "android-phone" 
                    WHEN device_id='1002' THEN "android-tablet" END AS device_code
                FROM df_ag   
            """)
            
            usage_df_age_gender.show()
            usage_df_age_gender.createOrReplaceTempView("df_ag")
            usage_df_age_gender_v2 = spark.sql("""
                SELECT df_ag.*,
                    est_active_users/est_usage_penetration AS est_population,
                    CAST(NULL AS DOUBLE) AS est_share_of_bytes_product_in_main_category,
                    est_active_users*est_average_time_per_user_milliseconds/60 AS est_total_time_minutes,
                    CAST(NULL AS DOUBLE) AS est_share_of_time_product_in_main_category,
                    est_active_users*est_average_session_per_user AS est_total_session_count,
                    CAST(NULL AS DOUBLE) AS est_share_of_session_product_in_main_category
                   
                FROM df_ag 
            """)
            usage_df_age_gender_v2.createOrReplaceTempView("df_ag")
            usage_df_age_gender_v2.show()

            usage_df_audience.show()
            usage_df_audience = usage_df_audience.groupby(usage_df_audience.device_id, usage_df_audience.store_id, usage_df_audience.date, usage_df_audience.app_id,usage_df_audience.gender,usage_df_audience.age).pivot("kpi").max("estimate")
            usage_df_audience = (
                usage_df_audience
                .withColumnRenamed('gender', 'gender_key')
                .withColumnRenamed('age', 'age_key')
                .withColumnRenamed('app_id', 'product_key')
                .withColumnRenamed('26', 'est_audience_index')
                .withColumnRenamed('27', 'est_audience_percentage')
                
            ) 
            
            usage_df_audience.createOrReplaceTempView("df_au")
            usage_df_audience = spark.sql("""
                SELECT 
                df_au.*,
                CASE WHEN device_id='1001' THEN "android-phone" 
                    WHEN device_id='1002' THEN "android-tablet" END AS device_code
                FROM df_au   
            """)
            usage_df_audience.show()
            usage_df_audience.createOrReplaceTempView("df_au")
            join_df=spark.sql("""
                SELECT ag.*,
                    au.est_audience_index,
                    au.est_audience_percentage
                FROM df_ag AS ag,df_au AS au
                WHERE ag.product_key=au.product_key
                AND ag.gender_key=au.gender_key
                AND ag.age_key=au.age_key
            """)
            join_df.createOrReplaceTempView("df")
            join_df.show()
            d -= datetime.timedelta(days=31)
            
            
            
params_list={
    'start':'2020-8-30',
    'end':'2020-8-31',
    'granularity':'monthly',
    'original_namespace_age_gender':'app-tech.usage.legacy-ag_app.v2',
    'original_namespace_audience':'app-tech.usage.legacy-ad_app.v2',
    'target_namespace':'aa.usage_demographics.v6'
    
}

main(spark,params=params_list)



In [0]:
%%sh
aws s3 ls s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_APP_CROSS_DOMAIN_METRICS/version=v1.0.0/granularity=MONTH/date=2019-01-31/platform=2/metric_name=CR/



In [0]:

df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.city.v6/fact/granularity=monthly/date=2020-01-31/")
df.count()
#df.groupBy('est_average_bytes_per_user').count().show()
#df.show(10,False)
url="s3://b2c-prod-data-pipeline-unified-usage/unified/usage.city.v6/fact/granularity=monthly/date=2020-01-31/"
spark.read.format("delta").load(url).count()

In [0]:

df = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.city-level.v3/fact").parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.city-level.v3/fact/granularity=m/date=2018-02-28/device_code=ap/country_code=us/part-00000-3f9dd3f5-f9fd-4b6a-a715-a44c1b48c339.c000.snappy.parquet")
df.createOrReplaceTempView("table1")

usage_df_age_gender = spark.sql("""
                SELECT 
                CASE WHEN granularity='m' THEN 1 END AS gran_value
                FROM table1
                """)
usage_df_age_gender.show()
#df.count()
#df.show(10,False)
#df.groupBy('metric_name').count().show()

In [0]:
%%sh
aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/